In [1]:
! pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, datediff, substring

In [ ]:
# conectado ao drive através da interface gráfica
'''
from google.colab import drive
drive.mount('/content/drive')
'''

In [3]:
spark = SparkSession.builder.getOrCreate()

In [5]:
clientes = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', header=True, inferSchema=True)

In [6]:
# Criar uma coluna com a classificação do NPS por avaliação

avaliacoes_pedido_nps_df = avaliacoes_pedido.withColumn(
    'NPS_classificacao',
    when(col('nota_avaliacao') == 5, 'Promotor')
    .when(col('nota_avaliacao') == 4, 'Neutro')
    .when(col('nota_avaliacao') <= 3, 'Detrator')
    .otherwise(None)
)

avaliacoes_pedido_nps_df.show()

+--------------------+--------------------+--------------+-----------------+--------------------+----------------------+-----------------------+-----------------+
|        id_avaliacao|           id_pedido|nota_avaliacao|titulo_comentario| mensagem_comentario|data_criacao_avaliacao|data_resposta_avaliacao|NPS_classificacao|
+--------------------+--------------------+--------------+-----------------+--------------------+----------------------+-----------------------+-----------------+
|7bc2406110b926393...|73fc7af87114b3971...|             4|             NULL|                NULL|   2018-01-18 00:00:00|    2018-01-18 21:46:59|           Neutro|
|80e641a11e56f04c1...|a548910a1c6147796...|             5|             NULL|                NULL|   2018-03-10 00:00:00|    2018-03-11 03:05:13|         Promotor|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|             5|             NULL|                NULL|   2018-02-17 00:00:00|    2018-02-18 14:36:24|         Promotor|
|e64fb393e7b32834b...|

In [7]:
# Calcular a nota do NPS

total_respostas = avaliacoes_pedido_nps_df.filter(col('nota_avaliacao').isNotNull()).count()
promotores = avaliacoes_pedido_nps_df.filter(col('NPS_classificacao') == 'Promotor').count()
detratores = avaliacoes_pedido_nps_df.filter(col('NPS_classificacao') == 'Detrator').count()

nps = ((promotores - detratores) / total_respostas) * 100 if total_respostas != 0 else 0

print(f'NPS: {nps:.2f}')

NPS: 33.97


In [9]:
# Mapear os clientes por Região
clientes_regiao_df = clientes.withColumn(
    'Regiao',
    when(col('estado_cliente').isin('AM', 'RR', 'AP', 'PA', 'AC', 'RO', 'TO'), 'Norte')
    .when(col('estado_cliente').isin('MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE'), 'Nordeste')
    .when(col('estado_cliente').isin('MT', 'MS', 'GO', 'DF'), 'Centro-Oeste')
    .when(col('estado_cliente').isin('SP', 'RJ', 'ES', 'MG'), 'Sudeste')
    .when(col('estado_cliente').isin('PR', 'SC', 'RS'), 'Sul')
    .otherwise('Desconhecido')
)

clientes_regiao_df.show()

+--------------------+--------------------+-----------+--------------------+--------------+-------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente| Regiao|
+--------------------+--------------------+-----------+--------------------+--------------+-------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|Sudeste|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|Sudeste|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|Sudeste|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|Sudeste|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|Sudeste|
|879864dab9bc30475...|4c93744516667ad3b...|      89254|      jaragua do sul|            SC|    Sul|
|fd826e7cf63160e53...|addec96d2e059c80c...|       4534|           sao paulo|            SP|Sudeste|


In [10]:
# Calcular o tempo decorrido entre a compra e a entrega
pedido_tempo_entrega_df = pedidos.withColumn('tempo_entrega', datediff(col('data_entrega_cliente'), col('data_compra_pedido')))
pedido_tempo_entrega_df.show()

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|tempo_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|            8|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|           14|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-0

In [11]:
# Criar uma coluna com o ano de entrega de cada pedido
pedido_ano_df = pedido_tempo_entrega_df.withColumn('ano_entrega', substring(col('data_compra_pedido'), 1,4))
pedido_ano_df.show()

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-------------+-----------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|tempo_entrega|ano_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+-------------+-----------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|            8|       2017|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|           14|       2018|
|47770eb9100c2d0c4...|41c

In [12]:
clientes_regiao_df.write.mode('overwrite').option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/clientes_preparada_parquet')
spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/output/clientes_preparada_parquet').show(5)

+--------------------+--------------------+-----------+--------------------+--------------+-------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente| Regiao|
+--------------------+--------------------+-----------+--------------------+--------------+-------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|Sudeste|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|Sudeste|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|Sudeste|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|Sudeste|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|Sudeste|
+--------------------+--------------------+-----------+--------------------+--------------+-------+
only showing top 5 rows



In [13]:
spark.stop()